In [17]:
import requests
import time
from tqdm import tqdm

SYSTEM_PROMPT = (
    "Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы."
)
MODEL_NAME = "t-tech/T-lite-it-1.0"

USER_PROMPT_MAIN = '''
Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки.

Запрос пользователя: {query}
Ответ:
'''

USER_PROMPT_POEM = '''
Ты — поэт-ассистент, вдохновляющийся классической русской поэзией.
На основе запроса пользователя сочини **новое** стихотворение в духе русской поэтической традиции.
Можешь использовать метафоры, эпитеты, ритм, рифму — всё, что придаёт тексту поэтичность.
Если знаешь строки из известных произведений — можешь вдохновляться ими, но **не копируй** дословно и не цитируй.
**НЕ ИСПОЛЬЗУЙ** в стихотворении имена и фамилии авторов.

Запрос пользователя: {query}
Стихотворение:
'''

def get_response_rag(query, is_poem):
    if is_poem:
        url = "http://localhost:8001/api/poem_async"
    else:
        url = "http://localhost:8001/api/generate_async"
    res = requests.post(
        url,
        json={"query": query},
        timeout=60
    )
    task_id = res.json()["task_id"]

    for _ in range(15):
        time.sleep(2)
        res = requests.get(f"http://localhost:8001/api/result/{task_id}")
        data = res.json()
        if data.get("status") == "completed":
            return data["result"]["response"]
    else:
        raise TimeoutError('ERROR')


def generate_sync_vllm(query, is_poem):
    if is_poem:
        prompt = USER_PROMPT_POEM.format(query=query)
    else:
        prompt = USER_PROMPT_MAIN.format(query=query)
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1024,
        "temperature": 0.7,
        "top_p": 0.9
    }
    try:
        response = requests.post('http://localhost:8000/v1/chat/completions', json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        fin_res = result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"[generate_sync] Ошибка запроса к vLLM: {e}")
        fin_res = ""
    
    if "." in fin_res:
        fin_res = fin_res.rsplit(".", 1)[0] + "."

    return fin_res


In [8]:
# QA

qa_queries = [
"Какие образы зимы использует Пушкин в своей поэзии?",
"Как авторы описывают образы грозы ?",
"Анализ стихотворения Анны Ахматовой 'А ты думал — я тоже такая'",
"Как Асадов раскрывает тему любви ?",
"Как поэты изображают осень в своих стихах?",
"Чем отличается стиль Есенина от стиля Мандельштама?",
"Как ритм и форма у Цветаевой отличаются от классической поэзии XIX века?",
"Особенности поэзии авторов Серебряного века",
"Основные литературные приемы, характерные для Фета",
"Почему Лермонтов описывает Родину противоречиво?"
]

In [5]:
qa_response_llm_lst = []
for q in tqdm(qa_queries):
    resp = generate_sync_vllm(q, is_poem=False)
    qa_response_llm_lst.append(resp)

100%|██████████| 10/10 [01:33<00:00,  9.34s/it]


In [14]:
# qa_queries[4:]

In [18]:
qa_response_rag_lst = []
for q in tqdm(qa_queries):
# for q in tqdm(qa_queries[4:]):
    resp = get_response_rag(q, is_poem=False)
    qa_response_rag_lst.append(resp)

100%|██████████| 10/10 [03:11<00:00, 19.19s/it]


In [ ]:
qa_response_llm_lst

['Александр Сергеевич Пушкин в своих произведениях мастерски использовал разнообразные образы зимы, создавая атмосферу морозного и волшебного времени года. Вот некоторые из них:\n\n1. **Снежный покров**: "Снега блестят под луной" (из поэмы "Медный всадник"). Здесь зима предстает как белоснежное одеяло, укрывающее землю.\n\n2. **Ледяная стужа**: "Вьюга воет, мчится буря, / Ломит ветви дубов..." (из стихотворения "Зимнее утро"). Зима здесь ассоциируется с холодной, бушующей природой.\n\n3. **Мороз и иней**: "Иней на ветвях сверкает, / А небо голубое..." (из стихотворения "Зимний вечер"). Это образы, которые создают ощущение хрупкости и чистоты зимней природы.\n\n4. **Заснеженные пейзажи**: "Зимний вечер. В доме тишина... / А за окном — снегопад..." (из стихотворения "Зимний вечер"). Здесь зима изображается как спокойное и умиротворяющее время.\n\n5. **Снежные дороги**: "По снежной равнине / Сквозь метели вихри..." (из стихотворения "Песнь о вещем Олеге"). Это образы путешествий и дорог, 

In [21]:
qa_response_rag_lst

['Александр Пушкин в своих стихотворениях создает яркие и многообразные образы зимы, передавая ее красоту и суровость. В "Зимнем утре" он описывает зимний пейзаж через контрасты: \n\n- **Свет и холод**: "Мороз и солнце; день чудесный!" — сочетание мороза и солнечного света создает уникальную атмосферу.\n- **Живая природа**: "Ель сквозь иней зеленеет," — несмотря на холод, природа сохраняет свою жизнь и красоту.\n- **Движение и покой**: "Блестя на солнце, снег лежит;" — статичность снега контрастирует с движением ветра и солнца.\n\nВ "2 ноября" Пушкин фокусируется на активности и радости зимних дней:\n\n- **Движение и приключения**: "Кружимся, рыскаем и поздней уж порой, / Двух зайцев протравив, являемся домой." — описание охоты и активного времяпрепровождения на свежем воздухе.\n- **Тепло и уют**: "Вся комната янтарным блеском / Озарена. Веселым треском / Трещит затопленная печь." — контраст между холодной зимой и теплом дома.\n\nВ "Зимней дороге" Пушкин передает чувство одиночества и 

In [23]:
import pandas as pd 

pd.DataFrame({
    'queries': qa_queries,
    'LLM': qa_response_llm_lst,
    'LLM_RAG': qa_response_rag_lst
}).to_csv('../data/ResponsesQA_judge.csv', index=False)


In [17]:
# print(qa_response_rag_lst[0])

In [16]:
# print(qa_response_rag_lst2[0])